In [1]:
import numpy as np
import pandas as pd

from torch.utils.data import DataLoader
import torch.utils.data as data_utils
import torch
from torch import nn
from torchvision import transforms, datasets, utils
from torch.utils.data import DataLoader, TensorDataset

# from catboost import CatBoostRegressor
# from lightgbm.sklearn import LGBMRegressor
# from xgboost import XGBRegressor
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.dummy import DummyRegressor

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import (
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor, export_graphviz

from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor

### Define scoring metrics and CV score function

In [2]:
scoring_metrics = {
    "neg RMSE": "neg_root_mean_squared_error",
}

In [3]:
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

### Load CSV files

In [4]:
df = pd.read_csv('../data/train.csv')
X_test_submit = pd.read_csv('../data/test.csv')

In [5]:
len(df)

75757

### Any manual feature engineering before column transformation

In [6]:
facility_class = pd.read_csv("f_type.csv")
facility_class["facility_class"].unique()

array(['Retail', 'Warehouse', 'Educational', 'Warehouse_cold', 'Office',
       'Flex_space', 'Commercial', 'Industrial', 'Public_Assembly',
       'Hotel', 'Health_care', 'Services', 'Food_services', 'Residential',
       'Public_safety'], dtype=object)

In [7]:
df = pd.merge(df, facility_class, on="facility_type")

df.head(3)

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id,facility_class
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,...,0,0,0,1.0,1.0,1.0,NaN,248.682615,0,Retail
1,1,State_1,Commercial,Grocery_store_or_food_market,67346.0,1967.0,26.0,1.8,36,50.5,...,0,0,0,1.0,NaN,1.0,12.0,287.863448,24,Retail
2,1,State_1,Commercial,Grocery_store_or_food_market,124196.0,1954.0,44.0,1.8,36,50.5,...,0,0,0,1.0,NaN,1.0,12.0,241.932986,25,Retail


In [8]:
df.shape

(75757, 65)

In [9]:
value = df["direction_max_wind_speed"]
df['dir_max_wind_speed'] = np.where(value > 337.5, "N",
                                np.where(value > 292.5, "NE",
                                        np.where(value > 247.5, "E",
                                                 np.where(value > 202.5, "SE",
                                                          np.where(value > 157.5, "S",
                                                                   np.where(value > 112.5, "SW",
                                                                            np.where(value > 67.5, "W",
                                                                                     np.where(value > 22.5, "NW", "N"))))))))

value = df["direction_peak_wind_speed"]
df['dir_peak_wind_speed'] = np.where(value > 337.5, "N",
                                np.where(value > 292.5, "NE",
                                        np.where(value > 247.5, "E",
                                                 np.where(value > 202.5, "SE",
                                                          np.where(value > 157.5, "S",
                                                                   np.where(value > 112.5, "SW",
                                                                            np.where(value > 67.5, "W",
                                                                                     np.where(value > 22.5, "NW", "N"))))))))

In [10]:
df.shape

(75757, 67)

In [11]:
df['dir_max_wind_speed'].unique()

array(['N', 'E', 'NE'], dtype=object)

In [12]:
df['dir_peak_wind_speed'].unique()

array(['N', 'NE', 'E'], dtype=object)

##### Checking the data I realized that the mean wind direction is 62 degrees which aligns with NE that we are getting above

In [13]:
# The following merges the imputed energy_star_rating
# This was done in R
df.to_csv("../data/train_facility.csv") ## export for R MICE imputation
energy_star_imp = pd.read_csv("energy_star_imp.csv")
df = pd.merge(df, energy_star_imp, on="facility_class")
df['energy_star_rating'] = df['energy_star_rating'].fillna(df.pop('energy_star_rating_imp'))

### Group columns for transformations

In [14]:
target = "site_eui"

numeric_features = [
    "floor_area", # Grouped and moved to ordinary feature
    "year_built",
    "energy_star_rating", # Imputed by facility_class + site_eui, take the average per facility_class
    # "ELEVATION", 
    "january_min_temp",
    "january_avg_temp",
    "january_max_temp",
#    "february_min_temp", # removed similar temperature columns
#    "february_avg_temp",
#    "february_max_temp",
#    "march_min_temp",
#    "march_avg_temp",
#    "march_max_temp",
#    "april_min_temp",
#    "april_avg_temp",
#    "april_max_temp",
#    "may_min_temp",
#    "may_avg_temp",
#    "may_max_temp",
#    "june_min_temp",
#    "june_avg_temp",
#    "june_max_temp",
    "july_min_temp",
    "july_avg_temp",
    "july_max_temp",
    "august_min_temp",
    "august_avg_temp",
    "august_max_temp",
#    "september_min_temp", # removed similar temperature columns
#    "september_avg_temp",
#    "september_max_temp",
#    "october_min_temp",
#    "october_avg_temp",
#    "october_max_temp",
#    "november_min_temp",
#    "november_avg_temp",
#    "november_max_temp",
#    "december_min_temp",
#    "december_avg_temp",
#    "december_max_temp",
    "cooling_degree_days",
    "heating_degree_days",
    "precipitation_inches",
    "snowfall_inches",
    "snowdepth_inches",
    "avg_temp",
   # "days_below_30F",
    "days_below_20F",
   # "days_below_10F", 
   # "days_below_0F",
   # "days_above_80F",
    "days_above_90F",
   # "days_above_100F",
   # "days_above_110F",
   # "direction_max_wind_speed",
   # "direction_peak_wind_speed",
    "max_wind_speed",
    "days_with_fog" ##???
]

ordinal_features = [] #['ord_floor_area']
categorical_features = [
                        "Year_Factor",  # Moved this down from numeric 
                        "State_Factor",
                        "facility_class",
                        "facility_type",
                        "dir_max_wind_speed",  # Added new feature
                        "dir_peak_wind_speed"]  # Added

drop_features = [
    "id",
    "building_class", # Moved this one here 
    #"floor_area", # Grouped and moved this one here 
    "direction_max_wind_speed",
    "direction_peak_wind_speed",
    "february_min_temp",
    "february_avg_temp",
    "february_max_temp",
    "march_min_temp",
    "march_avg_temp",
    "march_max_temp",
    "april_min_temp",
    "april_avg_temp",
    "april_max_temp",
    "may_min_temp",
    "may_avg_temp",
    "may_max_temp",
    "june_min_temp",
    "june_avg_temp",
    "june_max_temp",    
    "september_min_temp",
    "september_avg_temp",
    "september_max_temp",    
    "october_min_temp",
    "october_avg_temp",
    "october_max_temp",
    "november_min_temp",
    "november_avg_temp",
    "november_max_temp",
    "december_min_temp",
    "december_avg_temp",
    "december_max_temp",    
    "days_below_30F",    
    "days_below_10F",
    "days_below_0F",
    "days_above_80F",    
    "days_above_100F",
    "days_above_110F",    
    "ELEVATION", #Try dropping
]

assert df.columns.shape[0] == len(
    numeric_features
    + ordinal_features
    + categorical_features
    + [target]
    + drop_features
)

### Split data for CV

In [15]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=123)
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_test, y_test = test_df.drop(columns=[target]), test_df[target]

### Column transformation & preprocessors

In [16]:
numeric_transformer = make_pipeline(SimpleImputer(strategy="constant", fill_value=0), StandardScaler())

categorical_transformer = make_pipeline(
    OneHotEncoder(handle_unknown="ignore", sparse=True),
)

In [17]:
preprocessor = make_column_transformer(
    (numeric_transformer, numeric_features),
    (categorical_transformer, categorical_features),
    ("drop", drop_features),
)

### Check transformed df

In [18]:
X_train_transformed = preprocessor.fit_transform(X_train)

In [19]:
column_names = (
    numeric_features
    + preprocessor.named_transformers_["pipeline-2"]
    .named_steps["onehotencoder"]
    .get_feature_names_out(categorical_features)
    .tolist()
)

X_train_transformed_df = pd.DataFrame(
    X_train_transformed.toarray(), columns=column_names, index=X_train.index
)

X_train_transformed_df.head()

,floor_area,year_built,energy_star_rating,january_min_temp,january_avg_temp,january_max_temp,july_min_temp,july_avg_temp,july_max_temp,august_min_temp,...,facility_type_Warehouse_Nonrefrigerated,facility_type_Warehouse_Refrigerated,facility_type_Warehouse_Selfstorage,facility_type_Warehouse_Uncategorized,dir_max_wind_speed_E,dir_max_wind_speed_N,dir_max_wind_speed_NE,dir_peak_wind_speed_E,dir_peak_wind_speed_N,dir_peak_wind_speed_NE
21568,-0.363151,0.284266,-0.071249,-0.045100,0.021364,-0.008784,0.582177,0.509584,0.111418,0.560097,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
72973,0.137007,0.165142,-1.836260,-0.045100,0.021364,-0.008784,0.582177,0.509584,0.111418,0.560097,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
8015,-0.245682,-0.016854,0.083425,-0.045100,0.021364,-0.008784,0.582177,0.509584,0.111418,0.560097,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
15042,-0.441739,0.261103,-2.051506,-2.068464,-1.766486,-2.434134,-2.289493,-1.445458,-0.369150,-1.911152,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
12129,-0.577481,0.274339,0.187045,3.149685,2.970395,2.603133,-1.810881,-2.367796,-1.330285,-0.338539,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


### Dummy regressor as baseline

In [20]:
# results = {}
# pipe_dummy = DummyRegressor()
# results["Dummy"] = mean_std_cross_val_scores(
#     pipe_dummy, X_train, y_train, return_train_score=True, scoring=scoring_metrics
# )
# pd.DataFrame(results).T

### Train several models (CV) and retrieve the score

In [21]:
# pipe_ridge = make_pipeline(preprocessor, Ridge(random_state=123))

# pipe_rf = make_pipeline(
#     preprocessor, RandomForestRegressor(random_state=123, n_jobs=-1, max_depth=5)
# )

# pipe_xgb = make_pipeline(
#     preprocessor, XGBRegressor(random_state=123, n_jobs=-1, verbosity=0)
# )

# pipe_lgbm = make_pipeline(preprocessor, LGBMRegressor(random_state=123))

# pipe_catboost = make_pipeline(
#     preprocessor, CatBoostRegressor(random_state=123, verbose=0)
# )

# models = {
#     #"Ridge": pipe_ridge, ## high mse
#     #"Random Forest": pipe_rf,
#     "XGBoost": pipe_xgb,
#     "LightGBM": pipe_lgbm,
#     "CatBoost": pipe_catboost,
#     #"kNN": pipe_kNN,  ## high mse
# }

In [22]:
# for model_name, model in models.items():
#     results[model_name] = mean_std_cross_val_scores(
#         model, X_train, y_train, return_train_score=True, scoring=scoring_metrics
#     )

In [23]:
# pd.DataFrame(results).T

### Feature selection

### Hyperparameter tuning

## AutoML

## Averaging

In [24]:
# averaging_model = VotingRegressor(
#     list(models.items())
# )  # need the list() here for cross-validation to work!

In [25]:
# averaging_model.fit(X_train, y_train);

In [26]:
# results["Voting"] = mean_std_cross_val_scores(
#     averaging_model, X_train, y_train, return_train_score=True, scoring=scoring_metrics
# )

In [27]:
# pd.DataFrame(results)

## Stacking

In [28]:
# stacking_model = StackingRegressor(
#     list(models.items())
# )  # need the list() here for cross-validation to work!

In [29]:
# stacking_model.fit(X_train, y_train);

In [30]:
# results["Stacking"] = mean_std_cross_val_scores(
#     stacking_model, X_train, y_train, return_train_score=True, scoring=scoring_metrics
# )

### Test the selected model

In [31]:
# pipe = averaging_model

In [32]:
# pipe_fitted = pipe.fit(X_train, y_train)

In [33]:
# final_score = pipe.score(X_test, y_test)
# final_score

# AutoML prediction

# Neural Net

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device.type}")

Using device: cpu


In [35]:
# transform validation set
X_valid_transformed = preprocessor.transform(X_test)
X_valid_transformed_df = pd.DataFrame(
    X_valid_transformed.toarray(), columns=column_names, index=X_test.index
)

X_valid_t = torch.tensor(X_valid_transformed_df.to_numpy(np.float32), dtype=torch.float32)
y_valid_t = torch.tensor(y_test.to_numpy(np.float32), dtype=torch.float32)


# transform train set to tensor
X_t = torch.tensor(X_train_transformed_df.to_numpy(np.float32), dtype=torch.float32)
y_t = torch.tensor(y_train.to_numpy(np.float32), dtype=torch.float32)

print(X_t.shape)
print(y_t.shape)

torch.Size([68181, 116])
torch.Size([68181])


In [36]:
BATCH_SIZE = 128

train_dataset = TensorDataset(X_t, y_t)
valid_dataset = TensorDataset(X_valid_t, y_valid_t)

# DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [37]:
# train class
def trainer(model, device, criterion, optimizer, trainloader, validloader, epochs=5, verbose=True):
    """Simple training wrapper for PyTorch network."""
    
    train_loss = []
    valid_loss = []
    train_accuracy = []
    valid_accuracy = []
    
    for epoch in range(epochs):  # for each epoch
        
        train_batch_loss = 0
        train_batch_acc = 0
        valid_batch_loss = 0
        valid_batch_acc = 0
        
        # Training
        for X, y in trainloader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()       # Zero all the gradients w.r.t. parameters
            y_hat = model(X).flatten()
            loss = criterion(y_hat, y)   # Calculate loss based on output
            loss.backward()             # Calculate gradients w.r.t. parameters
            optimizer.step()            # Update parameters
            train_batch_loss += loss.item()  # Add loss for this batch to running total
            train_batch_acc += (torch.sqrt(torch.mean((y_hat-y)**2))*-1).type(torch.float32).item()   
            
        train_loss.append(train_batch_loss / len(trainloader))     # loss = total loss in epoch / number of batches = loss per batch
        train_accuracy.append(train_batch_acc / len(trainloader))  # accuracy
        
        # Validation
        model.eval()  # this turns off those random dropout layers, we don't want them for validation!
        
        with torch.no_grad():  # this stops pytorch doing computational graph stuff under-the-hood and saves memory and time
            for X, y in validloader:
                X, y = X.to(device), y.to(device)
                y_hat = model(X).flatten()  # Forward pass to get output
                loss = criterion(y_hat, y)   # Calculate loss based on output
                valid_batch_loss += loss.item()                  # Add loss for this batch to running total
                valid_batch_acc += (torch.sqrt(torch.mean((y_hat-y)**2))*-1).type(torch.float32).item()   # Average accuracy for this batch
                
        valid_loss.append(valid_batch_loss / len(validloader))
        valid_accuracy.append(valid_batch_acc / len(validloader))  # accuracy in terms of RMSE
        
        model.train()  # turn back on the dropout layers for the next training loop
        
        # Print progress
        if verbose:
            print(f"Epoch {epoch + 1:3}:",
                  f"Train Loss: {train_loss[-1]:.3f}.",
                  f"Valid Loss: {valid_loss[-1]:.3f}.",
                  f"Train Accuracy: {train_accuracy[-1]:.2f}.",
                  f"Valid Accuracy: {valid_accuracy[-1]:.2f}.")
    
    results = {"train_loss": train_loss,
               "valid_loss": valid_loss,
               "train_accuracy": train_accuracy,
               "valid_accuracy": valid_accuracy}
    return results

In [38]:
# NN Architecture
class site_model(torch.nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.main = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(32, 16),
            nn.BatchNorm1d(16),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(16, 8),
            nn.BatchNorm1d(8),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(8, 1),
        )
        
    def forward(self, x):
        return self.main(x)

In [39]:
# model to device
nn_model = site_model(X_t.shape[1])
nn_model.to(device)

# # Criterion and optimizer
# def RMSE(y_hat, y):
#     return torch.sqrt(torch.mean((y_hat - y) ** 2))

criterion = nn.L1Loss()
optimizer = torch.optim.Adam(nn_model.parameters(), lr=1e-3)

In [ ]:
training = trainer(nn_model, device, criterion, optimizer, train_loader, valid_loader, epochs=50)

Epoch   1: Train Loss: 80.166. Valid Loss: 75.990. Train Accuracy: -98.13. Valid Accuracy: -95.59.
Epoch   2: Train Loss: 67.766. Valid Loss: 57.309. Train Accuracy: -87.89. Valid Accuracy: -79.99.
Epoch   3: Train Loss: 45.327. Valid Loss: 33.448. Train Accuracy: -69.24. Valid Accuracy: -60.16.
Epoch   4: Train Loss: 30.781. Valid Loss: 25.319. Train Accuracy: -55.04. Valid Accuracy: -50.33.
Epoch   5: Train Loss: 27.852. Valid Loss: 23.727. Train Accuracy: -49.72. Valid Accuracy: -47.67.
Epoch   6: Train Loss: 27.276. Valid Loss: 22.943. Train Accuracy: -48.74. Valid Accuracy: -45.96.
Epoch   7: Train Loss: 27.130. Valid Loss: 22.806. Train Accuracy: -48.27. Valid Accuracy: -46.09.
Epoch   8: Train Loss: 26.870. Valid Loss: 22.961. Train Accuracy: -47.86. Valid Accuracy: -46.41.
Epoch   9: Train Loss: 26.745. Valid Loss: 22.544. Train Accuracy: -47.64. Valid Accuracy: -45.32.
Epoch  10: Train Loss: 26.633. Valid Loss: 22.961. Train Accuracy: -47.45. Valid Accuracy: -45.47.


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(np.array(training['train_loss']), label='train_loss')
plt.plot(np.array(training['valid_loss']), label='valid_loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend();

In [ ]:
plt.plot(np.array(training['train_accuracy']), label='train')
plt.plot(np.array(training['valid_accuracy']), label='valid')
plt.xlabel("Epoch")
plt.ylabel("RMSE")
plt.legend();

In [ ]:
X_test_submit['facility_type']

In [ ]:
# Transformation of test set
X_test_submit = pd.merge(X_test_submit, facility_class, on="facility_type")

# Impute energy_star_rating
X_test_submit = pd.merge(X_test_submit, energy_star_imp, on="facility_class")
X_test_submit['energy_star_rating'] = X_test_submit['energy_star_rating'].fillna(X_test_submit.pop('energy_star_rating_imp'))


value = X_test_submit["direction_max_wind_speed"]
X_test_submit['dir_max_wind_speed'] = np.where(value > 337.5, "N",
                                            np.where(value > 292.5, "NE",
                                                    np.where(value > 247.5, "E",
                                                             np.where(value > 202.5, "SE",
                                                                      np.where(value > 157.5, "S",
                                                                               np.where(value > 112.5, "SW",
                                                                                        np.where(value > 67.5, "W",
                                                                                                 np.where(value > 22.5, "NW", "N"))))))))

value = X_test_submit["direction_peak_wind_speed"]
X_test_submit['dir_peak_wind_speed'] = np.where(value > 337.5, "N",
                                            np.where(value > 292.5, "NE",
                                                    np.where(value > 247.5, "E",
                                                             np.where(value > 202.5, "SE",
                                                                      np.where(value > 157.5, "S",
                                                                               np.where(value > 112.5, "SW",
                                                                                        np.where(value > 67.5, "W",
                                                                                                 np.where(value > 22.5, "NW", "N"))))))))

#value_floor = X_test_submit["floor_area"]
#X_test_submit['ord_floor_area'] =  np.where(value_floor > 261980, 7,
#                              np.where(value_floor > 148466, 6,
#                                     np.where(value_floor > 105070, 5,
#                                            np.where(value_floor > 80088, 4,
#                                                  np.where(value_floor > 65333, 3,
#                                                         np.where(value_floor > 53250, 2, 1))))))

## Select your submission model

In [ ]:
X_test_transformed = preprocessor.transform(X_test_submit).toarray()
column_names_testing = (
    numeric_features
    + preprocessor.named_transformers_["pipeline-2"]
    .named_steps["onehotencoder"]
    .get_feature_names_out(categorical_features)
    .tolist()
)
X_test_transformed = pd.DataFrame(
   X_test_transformed, columns=column_names_testing)

In [ ]:
# State factor 6 needs to be manually added in
X_test_transformed['State_Factor_State_6'] = 0

# reorder columns to be in same order as training set
X_test_transformed = X_test_transformed[X_train_transformed_df.columns]

# Change to tensor
X_test_transformed= torch.tensor(X_test_transformed.values.astype(np.float32))

In [ ]:
# prediction = []
# for i in range(0,X_test_transformed.shape[0]):
#     prediction.append( model(X_test_transformed[i]).item())

prediction = nn_model(X_test_transformed).detach().numpy().flatten()
  
submission = pd.DataFrame({'id': X_test_submit['id'], 'site_eui': prediction})
submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)